# NLP Methods on Music Reviews: Feature Engineering


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
from numpy import dot
from numpy.linalg import norm
import math
import re
import sqlite3

import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/khyate/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Ingest Data

In [8]:
# read in reviews and baseline model features

reviews = pd.read_parquet("../datasets/reviews_w_sentiment.parquet").drop("score", axis=1)
baseline_df = pd.read_parquet("../datasets/baseline_features.parquet")

df = pd.concat([reviews, baseline_df], axis=1)
df.head()

,id,review,track,artist,url,best_new_music,author,author_type,date,weekday,day,month,year,preprocessed_review,Sentiment List (Pos/Neg/Neu),score,review_length,%Positive Sentiment,%Negative Sentiment,%Neutral Sentiment
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,0,nate patrin,contributor,2017-01-08,6,8,1,2017,trip-hop eventually became 90 punchline music-...,"[[best, famous, love, beautiful, top, sure, cr...",9.3,860.0,0.012791,0.003488,0.983721
1,22721,"Eight years, five albums, and two EPs in, the ...",prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,0,zoe camp,contributor,2017-01-07,5,7,1,2017,eight year five album two eps new york-based o...,"[[great, best, spontaneously, good], [grim, ha...",7.9,276.0,0.014493,0.010870,0.974638
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,0,david glickman,contributor,2017-01-07,5,7,1,2017,minneapolis uranium club seem revel aggressive...,"[[many, love, talented, perfect], [vaguely, ca...",7.3,326.0,0.012270,0.009202,0.978528
3,22661,Kleenex began with a crash. It transpired one ...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,kleenex began crash transpired one night long ...,"[[nice, delighted, ok, greatest, perfect, amus...",9.0,745.0,0.021477,0.002685,0.975839
4,22725,It is impossible to consider a given release b...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017,impossible consider given release footwork art...,"[[remarkable, best, warm, perfect, perfect, ab...",8.1,309.0,0.035599,0.003236,0.961165


# Feature Engineering

### Word Length Feature Count

In [9]:
def word_length(token_list):
    count = 0
    for token in token_list:
        if len(token) >= 6:
            count+=1
    return count


In [10]:
df["% Long Word Length"] = df['Sentiment List (Pos/Neg/Neu)'].apply(lambda x:(word_length(x[0]) + word_length(x[1]) + word_length(x[2])))/df['review_length']
df.head()

,id,review,track,artist,url,best_new_music,author,author_type,date,weekday,...,month,year,preprocessed_review,Sentiment List (Pos/Neg/Neu),score,review_length,%Positive Sentiment,%Negative Sentiment,%Neutral Sentiment,% Long Word Length
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,0,nate patrin,contributor,2017-01-08,6,...,1,2017,trip-hop eventually became 90 punchline music-...,"[[best, famous, love, beautiful, top, sure, cr...",9.3,860.0,0.012791,0.003488,0.983721,0.560465
1,22721,"Eight years, five albums, and two EPs in, the ...",prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,0,zoe camp,contributor,2017-01-07,5,...,1,2017,eight year five album two eps new york-based o...,"[[great, best, spontaneously, good], [grim, ha...",7.9,276.0,0.014493,0.010870,0.974638,0.576087
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,0,david glickman,contributor,2017-01-07,5,...,1,2017,minneapolis uranium club seem revel aggressive...,"[[many, love, talented, perfect], [vaguely, ca...",7.3,326.0,0.012270,0.009202,0.978528,0.561350
3,22661,Kleenex began with a crash. It transpired one ...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,1,jenn pelly,associate reviews editor,2017-01-06,4,...,1,2017,kleenex began crash transpired one night long ...,"[[nice, delighted, ok, greatest, perfect, amus...",9.0,745.0,0.021477,0.002685,0.975839,0.514094
4,22725,It is impossible to consider a given release b...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,0,kevin lozano,tracks coordinator,2017-01-06,4,...,1,2017,impossible consider given release footwork art...,"[[remarkable, best, warm, perfect, perfect, ab...",8.1,309.0,0.035599,0.003236,0.961165,0.543689


### Percent Common POS Tag

In [11]:
def POS_count(token_list):
    most_common_POS = ['NN', 'NNP', 'DT', 'IN', 'JJ', 'NNS','CC','PRP','VB','VBG']
    count = 0
    pos_tags = [x[1] for x in pos_tag(token_list)]
    for tag in pos_tags:
        if tag in most_common_POS:
            count +=1
    return count


In [12]:
df['% Common POS Tag'] = df['Sentiment List (Pos/Neg/Neu)'].apply(lambda x: POS_count(x[0]) + POS_count(x[1]) + POS_count(x[2]))/df['review_length']
df.head()

,id,review,track,artist,url,best_new_music,author,author_type,date,weekday,...,year,preprocessed_review,Sentiment List (Pos/Neg/Neu),score,review_length,%Positive Sentiment,%Negative Sentiment,%Neutral Sentiment,% Long Word Length,% Common POS Tag
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,0,nate patrin,contributor,2017-01-08,6,...,2017,trip-hop eventually became 90 punchline music-...,"[[best, famous, love, beautiful, top, sure, cr...",9.3,860.0,0.012791,0.003488,0.983721,0.560465,0.740698
1,22721,"Eight years, five albums, and two EPs in, the ...",prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,0,zoe camp,contributor,2017-01-07,5,...,2017,eight year five album two eps new york-based o...,"[[great, best, spontaneously, good], [grim, ha...",7.9,276.0,0.014493,0.010870,0.974638,0.576087,0.811594
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,0,david glickman,contributor,2017-01-07,5,...,2017,minneapolis uranium club seem revel aggressive...,"[[many, love, talented, perfect], [vaguely, ca...",7.3,326.0,0.012270,0.009202,0.978528,0.561350,0.797546
3,22661,Kleenex began with a crash. It transpired one ...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,1,jenn pelly,associate reviews editor,2017-01-06,4,...,2017,kleenex began crash transpired one night long ...,"[[nice, delighted, ok, greatest, perfect, amus...",9.0,745.0,0.021477,0.002685,0.975839,0.514094,0.783893
4,22725,It is impossible to consider a given release b...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,0,kevin lozano,tracks coordinator,2017-01-06,4,...,2017,impossible consider given release footwork art...,"[[remarkable, best, warm, perfect, perfect, ab...",8.1,309.0,0.035599,0.003236,0.961165,0.543689,0.796117


### NNP/NNPS tag

In [39]:
def proper_noun_percent(token_list):
    count = 0
    pos_tags = [x[1] for x in pos_tag(token_list)]
    for tag in pos_tags:
        if tag == 'NNP' or tag == 'NNPS':
            count +=1
    return count/len(token_list)

# test the function
print(proper_noun_percent(nltk.word_tokenize((df['review'][0]))))

0.13715570545250141


In [40]:
df['% Proper Noun'] = df['review'].apply(lambda x: proper_noun_percent(nltk.word_tokenize(x)))
df.head()

,id,review,track,artist,url,best_new_music,author,author_type,date,weekday,...,%Negative Sentiment,%Neutral Sentiment,% Long Word Length,% Common POS Tag,Proper Noun Count,Album mention counts,Artist mention counts,genre,best_sim_with_top_revs,% Proper Noun
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,0,nate patrin,contributor,2017-01-08,6,...,0.003488,0.983721,0.560465,0.740698,244,13,7,electronic,0.845,0.137156
1,22721,"Eight years, five albums, and two EPs in, the ...",prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,0,zoe camp,contributor,2017-01-07,5,...,0.010870,0.974638,0.576087,0.811594,59,3,4,metal,0.745,0.112595
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,0,david glickman,contributor,2017-01-07,5,...,0.009202,0.978528,0.561350,0.797546,64,1,9,rock,0.777,0.091691
3,22661,Kleenex began with a crash. It transpired one ...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,1,jenn pelly,associate reviews editor,2017-01-06,4,...,0.002685,0.975839,0.514094,0.783893,217,8,13,rock,0.791,0.135625
4,22725,It is impossible to consider a given release b...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,0,kevin lozano,tracks coordinator,2017-01-06,4,...,0.003236,0.961165,0.543689,0.796117,91,4,8,electronic,0.710,0.143082


### Artist/Album Mention

In [41]:
def percent_albarts_mention(token_list, track, artist):
    try:
        count1 = 0
        count2 = 0
        div1 = len(track.split())
        div2 = len(artist.split())
        for token in token_list:
            if token in track:
                count1+=1
        for token in token_list:
            if token in artist:
                count2+=1
        return math.ceil(count1/div1)/len(token_list), math.ceil(count2/div2)/len(token_list)
    except:
        return (0,0)
    
# test the function
print(percent_albarts_mention(df['Sentiment List (Pos/Neg/Neu)'][0][2], df['track'][0], df['artist'][0]))


(0.015366430260047281, 0.008274231678486997)


In [42]:
album_c = []
artist_c = []
for i, row in df.iterrows():
    al_c, ar_c = percent_albarts_mention(row['Sentiment List (Pos/Neg/Neu)'][2],row['track'],row['artist'])
    album_c.append(al_c)
    artist_c.append(ar_c)

df['% Album mention'] = album_c
df['% Artist mention'] = artist_c
df.head()

,id,review,track,artist,url,best_new_music,author,author_type,date,weekday,...,% Long Word Length,% Common POS Tag,Proper Noun Count,Album mention counts,Artist mention counts,genre,best_sim_with_top_revs,% Proper Noun,% Album mention,% Artist mention
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,0,nate patrin,contributor,2017-01-08,6,...,0.560465,0.740698,244,13,7,electronic,0.845,0.137156,0.015366,0.008274
1,22721,"Eight years, five albums, and two EPs in, the ...",prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,0,zoe camp,contributor,2017-01-07,5,...,0.576087,0.811594,59,3,4,metal,0.745,0.112595,0.011152,0.014870
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,0,david glickman,contributor,2017-01-07,5,...,0.561350,0.797546,64,1,9,rock,0.777,0.091691,0.003135,0.028213
3,22661,Kleenex began with a crash. It transpired one ...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,1,jenn pelly,associate reviews editor,2017-01-06,4,...,0.514094,0.783893,217,8,13,rock,0.791,0.135625,0.011004,0.017882
4,22725,It is impossible to consider a given release b...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,0,kevin lozano,tracks coordinator,2017-01-06,4,...,0.543689,0.796117,91,4,8,electronic,0.710,0.143082,0.013468,0.026936


### Most similar from the top cosine similarities of each genre

In [28]:
connection = sqlite3.connect('../datasets/database.sqlite')
cursor = connection.cursor()

query = "select * from genres;"
cursor.execute(query)
df_genres = pd.DataFrame(cursor.fetchall(), columns=["reviewid","genre"])
df_genres.head()

,reviewid,genre
0,22703,electronic
1,22721,metal
2,22659,rock
3,22661,rock
4,22725,electronic


In [29]:
# inspect distribution

df_genres['genre'].value_counts()

rock            9436
electronic      3874
experimental    1815
rap             1559
pop/r&b         1432
metal            860
folk/country     685
jazz             435
global           217
Name: genre, dtype: int64

In [30]:
# add Genre column by joining tables

df_genres.rename(columns={"reviewid": "id"}, inplace=True)
b_dict = dict(zip(df_genres['id'], df_genres['genre']))
df['genre'] = df['id'].map(b_dict)
df.head()

,id,review,track,artist,url,best_new_music,author,author_type,date,weekday,...,review_length,%Positive Sentiment,%Negative Sentiment,%Neutral Sentiment,% Long Word Length,% Common POS Tag,Proper Noun Count,Album mention counts,Artist mention counts,genre
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,0,nate patrin,contributor,2017-01-08,6,...,860.0,0.012791,0.003488,0.983721,0.560465,0.740698,244,13,7,electronic
1,22721,"Eight years, five albums, and two EPs in, the ...",prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,0,zoe camp,contributor,2017-01-07,5,...,276.0,0.014493,0.010870,0.974638,0.576087,0.811594,59,3,4,metal
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,0,david glickman,contributor,2017-01-07,5,...,326.0,0.012270,0.009202,0.978528,0.561350,0.797546,64,1,9,rock
3,22661,Kleenex began with a crash. It transpired one ...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,1,jenn pelly,associate reviews editor,2017-01-06,4,...,745.0,0.021477,0.002685,0.975839,0.514094,0.783893,217,8,13,rock
4,22725,It is impossible to consider a given release b...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,0,kevin lozano,tracks coordinator,2017-01-06,4,...,309.0,0.035599,0.003236,0.961165,0.543689,0.796117,91,4,8,electronic


# Get similarity to the "Best" reviews for each Genre

In [33]:
def first_of_best(genres):
    ''' getting the first review from those with 10/10 score of each genre.'''
    no_null = df.dropna(axis=0, subset="genre")
    ids = []
    for g in genres:
        print(g)
        if g == 'metal':
            ids.append(no_null.loc[(df['score'] == 9.7) & (no_null['genre'] == g)].loc[:,'id'].iloc[0])
        elif g == 'global':
            ids.append(no_null.loc[(df['score'] == 9.4) & (no_null['genre'] == g)].loc[:,'id'].iloc[0])
        else:
            ids.append(no_null.loc[(df['score'] == 10) & (no_null['genre'] == g)].loc[:,'id'].iloc[0])
    print(f"ID's of Best Reviews for each Genre: {ids}")
    return ids

genres = df['genre'].value_counts().index
best_ids = first_of_best(genres)
    

rock
electronic
experimental
rap
pop/r&b
metal
folk/country
jazz
global
ID's of Best Reviews for each Genre: [22374, 2377, 22061, 21218, 22174, 8383, 699, 21158, 22255]


In [34]:
# calculate cosine similarity between each review and the "Best" reviews

bests = df[df['id'].isin(best_ids)]
best_reviews = list(bests.loc[:,'review'])

df['best_sim_with_top_revs'] = df['review'].apply(cossims)
df.head()

,id,review,track,artist,url,best_new_music,author,author_type,date,weekday,...,%Positive Sentiment,%Negative Sentiment,%Neutral Sentiment,% Long Word Length,% Common POS Tag,Proper Noun Count,Album mention counts,Artist mention counts,genre,best_sim_with_top_revs
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,0,nate patrin,contributor,2017-01-08,6,...,0.012791,0.003488,0.983721,0.560465,0.740698,244,13,7,electronic,0.845
1,22721,"Eight years, five albums, and two EPs in, the ...",prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,0,zoe camp,contributor,2017-01-07,5,...,0.014493,0.010870,0.974638,0.576087,0.811594,59,3,4,metal,0.745
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,0,david glickman,contributor,2017-01-07,5,...,0.012270,0.009202,0.978528,0.561350,0.797546,64,1,9,rock,0.777
3,22661,Kleenex began with a crash. It transpired one ...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,1,jenn pelly,associate reviews editor,2017-01-06,4,...,0.021477,0.002685,0.975839,0.514094,0.783893,217,8,13,rock,0.791
4,22725,It is impossible to consider a given release b...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,0,kevin lozano,tracks coordinator,2017-01-06,4,...,0.035599,0.003236,0.961165,0.543689,0.796117,91,4,8,electronic,0.710


In [46]:
def cossims(text):
    sims = []
    vectorizer = TfidfVectorizer()
    for rev in best_reviews:
        vectors = vectorizer.fit_transform([text,rev])
        sims.append(cosine_similarity(vectors)[0][1])
    return round(max(sims),3)

# test the function
print(cossims(df['review'][3]))

0.791


## NLTK Sentiment Analyzer

In [63]:

analyzer = SentimentIntensityAnalyzer()

def pos_sentiment(text):

    scores = analyzer.polarity_scores(text)
    return scores["pos"]

def neg_sentiment(text):

    scores = analyzer.polarity_scores(text)
    return scores["neg"]

def neu_sentiment(text):

    scores = analyzer.polarity_scores(text)
    return scores["neu"]

df['nltk_pos'] = df['review'].apply(pos_sentiment)
df['nltk_neg'] = df['review'].apply(neg_sentiment)
df['nltk_neu'] = df['review'].apply(neu_sentiment)
df.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/khyate/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,id,review,track,artist,url,best_new_music,author,author_type,date,weekday,...,% Long Word Length,% Common POS Tag,genre,best_sim_with_top_revs,% Proper Noun,% Album mention,% Artist mention,nltk_pos,nltk_neg,nltk_neu
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,0,nate patrin,contributor,2017-01-08,6,...,0.560465,0.740698,electronic,0.845,0.137156,0.015366,0.008274,0.098,0.101,0.800
1,22721,"Eight years, five albums, and two EPs in, the ...",prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,0,zoe camp,contributor,2017-01-07,5,...,0.576087,0.811594,metal,0.745,0.112595,0.011152,0.014870,0.053,0.077,0.870
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,0,david glickman,contributor,2017-01-07,5,...,0.561350,0.797546,rock,0.777,0.091691,0.003135,0.028213,0.086,0.049,0.865
3,22661,Kleenex began with a crash. It transpired one ...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,1,jenn pelly,associate reviews editor,2017-01-06,4,...,0.514094,0.783893,rock,0.791,0.135625,0.011004,0.017882,0.148,0.077,0.776
4,22725,It is impossible to consider a given release b...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,0,kevin lozano,tracks coordinator,2017-01-06,4,...,0.543689,0.796117,electronic,0.710,0.143082,0.013468,0.026936,0.127,0.012,0.862


# Save Features

In [36]:
# Drop reviews with null review length

df.drop(df[df['review_length'].isna()].index, inplace=True)

In [64]:
# df.to_parquet("../datasets/full_engineered_features.parquet")
df.columns

Index(['id', 'review', 'track', 'artist', 'url', 'best_new_music', 'author',
       'author_type', 'date', 'weekday', 'day', 'month', 'year',
       'preprocessed_review', 'Sentiment List (Pos/Neg/Neu)', 'score',
       'review_length', '%Positive Sentiment', '%Negative Sentiment',
       '%Neutral Sentiment', '% Long Word Length', '% Common POS Tag', 'genre',
       'best_sim_with_top_revs', '% Proper Noun', '% Album mention',
       '% Artist mention', 'nltk_pos', 'nltk_neg', 'nltk_neu'],
      dtype='object')

In [65]:
# Isolate the features just for modeling

df_model = df[["score","review_length", "%Positive Sentiment", "%Negative Sentiment", "%Neutral Sentiment",
"% Long Word Length", "% Common POS Tag", '% Proper Noun', '% Album mention',
       '% Artist mention', "best_sim_with_top_revs", 'nltk_pos', 'nltk_neg', 'nltk_neu']]

# df_model.to_parquet("../datasets/engineered_features.parquet")
df_model

,score,review_length,%Positive Sentiment,%Negative Sentiment,%Neutral Sentiment,% Long Word Length,% Common POS Tag,% Proper Noun,% Album mention,% Artist mention,best_sim_with_top_revs,nltk_pos,nltk_neg,nltk_neu
0,9.3,860.0,0.012791,0.003488,0.983721,0.560465,0.740698,0.137156,0.015366,0.008274,0.845,0.098,0.101,0.800
1,7.9,276.0,0.014493,0.010870,0.974638,0.576087,0.811594,0.112595,0.011152,0.014870,0.745,0.053,0.077,0.870
2,7.3,326.0,0.012270,0.009202,0.978528,0.561350,0.797546,0.091691,0.003135,0.028213,0.777,0.086,0.049,0.865
3,9.0,745.0,0.021477,0.002685,0.975839,0.514094,0.783893,0.135625,0.011004,0.017882,0.791,0.148,0.077,0.776
4,8.1,309.0,0.035599,0.003236,0.961165,0.543689,0.796117,0.143082,0.013468,0.026936,0.710,0.127,0.012,0.862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18396,8.9,280.0,0.017857,0.007143,0.975000,0.439286,0.778571,0.111693,0.025641,0.036630,0.657,0.103,0.042,0.855
18397,4.8,134.0,0.044776,0.014925,0.940299,0.432836,0.649254,0.039427,0.007937,0.007937,0.571,0.207,0.033,0.760
18398,6.3,457.0,0.019694,0.019694,0.960613,0.382932,0.684902,0.097585,0.002278,0.009112,0.731,0.135,0.094,0.771
18399,7.2,243.0,0.024691,0.008230,0.967078,0.456790,0.699588,0.052301,0.004255,0.004255,0.580,0.208,0.097,0.695
